Задание к лекции "Основы веб-скрапинга и работы с API"

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [12]:
import pandas as pd
import requests
import time
import regex
from bs4 import BeautifulSoup
url = 'https://habr.com/ru/all/'
keywords = ['Python', 'python', 'Парсинг','парсинг']


In [15]:
def get_all_info(url, keywords, pages):
    habr_posts = pd.DataFrame()
    
    for i in range(pages):
        params = 'page' + str(i + 1)
        res = requests.get(url+params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        
        title_blocks = soup.find_all('h2', class_='post__title')
        text_blocks = soup.find_all('div', class_='post__text post__text-html post__text_v1')
        time_block = soup.find_all('span', class_='post__time')
        
        all_titles = list(map(lambda x: x.find('a', class_='post__title_link').text, title_blocks))
        all_linkes = list(map(lambda x: x.find('a').get('href'), title_blocks))
        all_text = list(map(lambda x: x.get_text(),text_blocks))
        all_time = list(map(lambda x: x.get_text(), time_block))
        all_info = [list(tup) for tup in zip(all_time, all_titles, all_text, all_linkes)]
        
        for j in range(len(all_info)):
            if regex.search(r"\L<words>", all_info[j][1], words=keywords) or\
            regex.search(r"\L<words>", all_info[j][2], words=keywords):
                row = {'дата': all_info[j][0], 'заголовок': all_info[j][1], 'ссылка': all_info[j][3], 'текст статьи': all_info[j][2]}
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])]) 
                 
    return habr_posts

all_links = get_all_info(url, keywords, 30)
all_links

,дата,заголовок,ссылка,текст статьи
0,сегодня в 16:56,Как строить красивые графики на Python с Seaborn,https://habr.com/ru/company/otus/blog/540526/,\nДве команды исследователей нашли разные спос...
0,сегодня в 12:18,Почему в InVision затаскивают микросервисы обр...,https://habr.com/ru/company/flant/blog/540406/,"\n Привет, Хаброжители! Код высокого качества ..."
0,вчера в 11:14,Книга «Теоретический минимум. Специальная теор...,https://habr.com/ru/company/piter/blog/538468/,"\nБывают ситуации, когда нужно упаковать Pytho..."
0,вчера в 10:42,Упаковка любого python пакета в rpm пакет с во...,https://habr.com/ru/post/540246/,"\n\n\r\nПривет, Хабр! Ещё до продажи Мосигры м..."
0,вчера в 10:10,Разбор популярных книг по Python,https://habr.com/ru/post/540030/,"\nПривет! 2020 позади, но запомнится нам надол..."
0,31 января 2021 в 22:29,Автоматизируем поиск секретов в git и ansible,https://habr.com/ru/post/540190/,"\n В этом цикле уроков я расскажу о том, ..."
0,31 января 2021 в 20:25,Уроки компьютерного зрения на Python + OpenCV ...,https://habr.com/ru/post/540166/,\n\n\r\nВ 2012 году в структуре Amazon возникл...
0,29 января 2021 в 15:46,Vanilla All the Way. Ванильное облачное решени...,https://habr.com/ru/company/timeweb/blog/539944/,"\n\n\r\nТе, кто работает с Python, знают, что ..."
0,29 января 2021 в 15:12,10 удивительно полезных базовых функций Python,https://habr.com/ru/company/skillbox/blog/539932/,"\n\n\r\nОглядываясь назад, моя статья по повод..."
0,29 января 2021 в 11:10,Читкоды для рынка найма 2021: прогнозы для раб...,https://habr.com/ru/post/539868/,\n Что связывает языки Python и C++? Как ...
